In [1]:
import vectorbt as vbt
import pandas as pd
from datetime import datetime
import numpy as np
from numba import njit
from tqdm import tqdm

import plotly.graph_objects as go
import plotly.express as px
import typing as tp

# CONST

In [2]:
BTC = '../../btc_data/BTC.pck'
# streamlit_yfinance/btc_data/BTC.pck
# streamlit_yfinance/work_folder_temp/BTC_S2F_TESTS/vectorbt.ipynb

# DEfs

In [3]:
def get_BTC():
    df = pd.read_pickle(BTC)
    return df
    

In [4]:
def add_volatility(df, vol_windows=[3, 7, 10, 20, 30]):

    
    for w in vol_windows:

        mean_col = f'mean_{w}'
        std_col = f'std_{w}'
        CV_col = f'CV_{w}'
        
        df[mean_col] = df['BTC'].rolling(window=30, center=True).mean()
        df[std_col] = df['BTC'].rolling(window=30, center=True).std()
        
        # Współczynnik zmienności
        df[CV_col] = (df[std_col] / df[mean_col]) * 100  # Współczynnik zmienności w %

        df.drop(columns=[mean_col, std_col], inplace=True)

       
    df.dropna(inplace=True)
    return df

def add_pct_change(df, days_ptc_change = [1, 3, 7, 15]):
    # add pct_change Future and Past
    
    # for d in days_ptc_change:
    #     df[f'p{str(d)}'] = df.BTC.pct_change(d)   #pct change past
    
    for d in days_ptc_change:
        df[f'f{str(d)}'] = df.BTC.pct_change(-d)  #pct change future
    
    all_data_cols = df.columns[3:]
    
    df.dropna(inplace=True)
    return df

# MAIN

In [5]:
df = get_BTC()
df.index = df.index.astype('datetime64[ns]')
df.head(3)

,dfh,e,BTC
2010-07-18,561.0,1,0.04951
2010-07-19,562.0,1,0.08584
2010-07-20,563.0,1,0.08080


In [6]:


# # Przykładowe dane - kurs BTC
# dates = pd.date_range(start='2023-01-01', periods=100)
# prices = pd.Series([30000 + i * 100 for i in range(100)], index=dates)  # Ceny BTC
# df = pd.DataFrame({'BTC': prices})

# Dodajmy kolumnę BUYSIG - sygnał kupna (dla przykładu, kupujemy co 10 dni)
df['BUYSIG'] = [True if i % 10 == 0 else False for i in range(len(df))]
df['SELLSIG'] = False
# Parametry strategii
# stop_loss = 0.05  # 5% stop loss
# take_profit = 0.1  # 10% take profit
df.head(3)

,dfh,e,BTC,BUYSIG,SELLSIG
2010-07-18,561.0,1,0.04951,True,False
2010-07-19,562.0,1,0.08584,False,False
2010-07-20,563.0,1,0.08080,False,False


In [7]:
df['BUYSIG'].value_counts()

BUYSIG
False    4707
True      523
Name: count, dtype: int64

# VECTOR

In [25]:

@njit
def adjust_sl(c:vbt.portfolio.AdjustSLContext, sl_percentage:0.05):
    price = c.curr_price  # Bieżąca cena
    stop_loss = c.curr_stop  # Bieżący stop loss
    trailing = False  # Flaga trailing

    # Ustawienie stop loss
    if price > stop_loss:
        stop_loss = price * (1 - sl_percentage)  # Ustawienie nowego stop loss

    
    return stop_loss, trailing

In [26]:
SL = 0.05  # 5% początkowy Stop Loss
ACT = 0.20  # Aktywacja trailing SL po wzroście o 20% od SL
TSL = 0.03  # Trailing Stop Loss na poziomie 3% poniżej maksimum ceny
sl_percentage = 0.05  # 5% stop loss

In [27]:
# Przeprowadzamy backtest na podstawie kolumny BUYSIG
# vbt.Portfolio.from_signals wymaga cen, sygnałów kupna i sprzedaży
entries = df['BUYSIG']  # Sygnały kupna
exits = df['SELLSIG']   # Nie ma z góry określonych sygnałów sprzedaży

# Tworzymy portfel z parametrami StopLoss i TakeProfit
portfolio = vbt.Portfolio.from_signals(
    
    close=df['BTC'],  # Ceny zamknięcia
    entries=entries,  # Sygnały wejścia
    exits=exits,  # Sygnały wyjścia
    sl_stop=SL,  # StopLoss w postaci procentowej
    adjust_sl_func_nb=adjust_sl,  # Nasza funkcja do dynamicznego SL
    aadjust_sl_args=(sl_percentage),
    size=100
)


TypeError: missing a required argument: 'sl_percentage'

In [ ]:
# Wizualizacja wyników
portfolio.plot().show()

# Wyświetlenie podsumowania wyników
print(portfolio.stats())

print(portfolio.stats())
print(portfolio.orders.records_readable)